In [19]:
import os
import geopandas as gpd
import pandas as pd
from shapely import wkt
import glob
from typing import Optional, Union, Dict, List, Any

In [ ]:
def csv_conversion(csv_path, crs='EPSG:4326', dtypes=None):
    """
    Convert a CSV file to a geodataframe
    """
    df = pd.read_csv(csv_path, dtype=dtypes)
    # Convert the geometry column to a geodataframe
    gdf = gpd.GeoDataFrame(df, geometry=df['geometry'].apply(wkt.loads), crs=crs)

    return gdf
    

In [51]:
csv_path = r'D:\Projects\superparcels\data\abreunig_pocs_spatialrecord_polygon_superparcel_urban.csv'
output_dir = r'D:\Projects\superparcels\data\Urban'
county_names = {
    '06075': 'San_Fran_CA',
    '06001': 'Alameda_CA',
    '08031': 'Denver_CO',
    '48113': 'Dallas_TX',
}

In [9]:
df = csv_conversion(csv_path, dtype={'FIPS': str})
fips = df['FIPS'].unique()


In [11]:
for fid in fips:
    print(fid)
    df_fid = df[df['FIPS'] == fid]
    fid_dir = os.path.join(output_dir, county_names[fid])
    if not os.path.exists(fid_dir):
        os.makedirs(fid_dir)

    df_fid.to_file(os.path.join(fid_dir, f'sp_sample_{fid}.shp'))

array(['06075', '08031', '06001', '48113'], dtype=object)

### Remove duplicate geometries and find canidate parcels for clustering POC

In [34]:

def classify_duplicates(df, fields):
    """
    Classifies the rows of a DataFrame based on duplicate status for any number of fields.

    Parameters:
    df (pd.DataFrame): DataFrame containing the data.
    fields (list of str): List of column names to check for duplicates.

    Returns:
    pd.Series: Value counts of each classification.
    """
    # Identify duplicates for each field and store in a new column
    for field in fields:
        df[f'duplicate_{field}'] = df.duplicated(subset=[field], keep=False)

    # Create a classification column based on the duplication status of each field
    def classify_row(row):
        classification = []
        for field in fields:
            if row[f'duplicate_{field}']:
                classification.append(f'Duplicate {field}')
            else:
                classification.append(f'Unique {field}')
        return ', '.join(classification)

    df['classification'] = df.apply(classify_row, axis=1)
    df['classify_codes'] = df['classification'].astype('category').cat.codes

    # Count the occurrences of each classification
    return df

# Example usage:
# fields = ['OWNER', 'std_addr', 'geometry']
# matrix = classify_duplicates(df, fields)
# print(matrix)


In [41]:
def codes_to_shp(df, id_field: str, code_field: str, cluster_canidate_codes: Optional[List] = []):
    """
    Convert a DataFrame to a shapefile based on classification codes.

    Parameters:
    df (pd.DataFrame): DataFrame containing the data.
    id_field (str): Name of the field to use as the unique identifier.
    code_field (str): Name of the field containing the classification codes.
    cluster_canidate_codes (list of int): List of classification codes to use as cluster candidates.
    """

    for code in class_codes:
        gdf = df[df[code_field] == code]
        gdf.to_file(os.path.join(data_dir, f'sp_sample_{id_field}_{code}.shp'))

    for code in cluster_canidate_codes:
        gdf = df[df[code_field] == code]
        print(f'Cluster canidates from code {code}: {gdf['classification'].unique()}')
        gdf.to_file(os.path.join(data_dir, f'sp_sample_{id_field}_cluster_candidates.shp'))

    return

In [52]:
for fid in fips:
    print(f'Processing {fid}')
    county_name = county_names[fid]
    file_path = os.path.join(output_dir, f'{county_name}\sp_sample*.shp')

    if len(glob.glob(file_path)) == 0:
        print(file_path)
        continue

    df = gpd.read_file(fi)

    df = classify_duplicates(df, fields=['OWNER', 'geometry'])
    print(df['classification'].value_counts())
    print(df['classify_codes'].value_counts())

    canidate_df = df[df['classification'] == 'Duplicate OWNER, Unique geometry']

    codes_to_shp(df, id_field='FIPS', code_field='classify_codes', cluster_canidate_codes=canidate_df['classify_codes'].unique())
    print('_____________________________')

06075
classification
Unique OWNER, Unique geometry          65119
Unique OWNER, Duplicate geometry       31794
Duplicate OWNER, Unique geometry       17973
Duplicate OWNER, Duplicate geometry     6132
Name: count, dtype: int64
classify_codes
1    17973
Name: count, dtype: int64
